In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from scipy.stats import spearmanr
import re
import arviz as az
import pymc as pm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import cellbayesassay as cba
import pickle
import re
import attila_utils

In [2]:
fpath = '../../resources/cell-based-assays/CO28151_outcome-summary_27062023-clean.xlsx'
conc2concentration = pd.read_excel(fpath, 1).rename({'TI nr.': 'TI'}, axis=1)
cN_columns = [c for c in conc2concentration.columns if re.match('^c[1-6]$', c)]
other_columns = [c for c in conc2concentration.columns if not re.match('^c[1-6]$', c)]
conc2concentration = pd.concat([conc2concentration[other_columns], conc2concentration[cN_columns].applymap(lambda x: x * 1e-6)], axis=1)
conc2concentration['VC'] = conc2concentration.apply(lambda r: r.loc['c6'] * 1e-3 if not np.isnan(r.loc['c6']) else r.loc['c3'] * 1e-3, axis=1)
conc2concentration['LPS'] = conc2concentration['VC']
conc2concentration = conc2concentration.set_index('TI')
conc2concentration

,TI ID,name,c1,c2,c3,c4,c5,c6,VC,LPS
TI,,,,,,,,,,
TI1,2542,Amiloride,0.000100,0.000010,1.000000e-06,NaN,NaN,NaN,1.000000e-09,1.000000e-09
TI2,2543,Phenamil methanesulfonate salt,0.000100,0.000010,1.000000e-06,NaN,NaN,NaN,1.000000e-09,1.000000e-09
TI3,2538,Benzamil hydrochloride hydrate,0.000100,0.000010,1.000000e-06,NaN,NaN,NaN,1.000000e-09,1.000000e-09
TI4,2544,"5-(N,N-Dimethyl)amiloride hydrochloride",0.000100,0.000010,1.000000e-06,NaN,NaN,NaN,1.000000e-09,1.000000e-09
TI5,2540,5-(N-Ethyl-N-isopropyl)amiloride,0.000100,0.000010,1.000000e-06,NaN,NaN,NaN,1.000000e-09,1.000000e-09
TI6,2541,"5-(N,N-Hexamethylene)amiloride",0.000010,0.000001,2.000000e-07,NaN,NaN,NaN,2.000000e-10,2.000000e-10
TI7,2539,5-(N-Methyl-N-isobutyl)amiloride,0.000100,0.000010,1.000000e-06,NaN,NaN,NaN,1.000000e-09,1.000000e-09
TI8,2719,NS 383,0.000100,0.000010,1.000000e-06,NaN,NaN,NaN,1.000000e-09,1.000000e-09
TI9,2703,A-317567,0.000100,0.000010,1.000000e-06,NaN,NaN,NaN,1.000000e-09,1.000000e-09


In [3]:
control_TIl = [
    'VC',
    'VC',
    'VC',
    'LPS',
    'VC'
]

experimentl = [
    'Aβ clearance (BV2 cells)',
    'Aβ release (H4 cells)',
    'Trophic factor withdrawal (primary neurons)',
    'LPS neuroinflammation (BV2 cells)',
    'Tau phoshorylation'
              ]
experimentd = dict(zip(experimentl, np.arange(len(experimentl))))
treatmentd = {
    'TI11': 'DIM-C-pPhOCH3',
    'TI12': 'C-DIM12',
}
fpath = '../../resources/cell-based-assays/CO28151_data-summary-clean.xlsx'

In [4]:
def extract_TI_exp_data(experiment, TI, control_TI='VC', batchvars=['Batch', 'Plate'], experimentd=experimentd, fpath=fpath):
    data = pd.read_excel(fpath, experimentd[experiment])
    TI_data = data.loc[data.TI == TI]
    # ensure that all treatment TI data are from the same batch:plate
    if len(TI_data.groupby(batchvars)) != 1:
        print('treatment with multiple batches')
        return(None)
    b = data.Plate == TI_data.iloc[0].loc['Plate']
    # if there's no information on the
    if not TI_data.iloc[0].isna().loc['Batch']:
        b = b & (data.Batch == data.iloc[0].loc['Batch'])
    TI_exp_data = data.loc[b]
    TI_exp_data_control = TI_exp_data.loc[TI_exp_data.TI == control_TI].copy()
    # if there's no control for the same batch:plate, use controls from all other batch:plate combinations
    if len(TI_exp_data_control) == 0:
        TI_exp_data_control = data.loc[data.TI == control_TI].copy()
    TI_exp_data_control['conc'] = control_TI
    TI_exp_data_TI = TI_exp_data.loc[TI_exp_data.TI == TI].copy()
    TI_exp_data = pd.concat([TI_exp_data_control, TI_exp_data_TI], axis=0)
    return(TI_exp_data)

experiment = experimentl[0]
TI = list(treatmentd.keys())[0]

def add_concentrations(df, conc2concentration=conc2concentration):
    df['concentration'] = df.conc.apply(lambda x: conc2concentration.loc[TI, x])
    return(df)

def process_TI_list_exp(experiment, control_TI='VC', TI_list=['TI11', 'TI12'], conc2concentration=conc2concentration):
    d = {TI: add_concentrations(extract_TI_exp_data(experiment, TI, control_TI)) for TI in TI_list}
    return(d)

In [5]:
datad = {experiment: process_TI_list_exp(experiment, control_TI=cTI) for experiment, cTI in zip(experimentl, control_TIl)}
datad[experimentl[0]]['TI11']

,Batch,Plate,Group,IRN,TI,conc,Aβ42 SN,Aβ42 Lys,BCA,Aβ42 Lys pg per ug prot,concentration
205,NaN,1.0,VC1,VC_1_1,VC,VC,4923,740.0,440.0,1.68,2.000000e-10
206,NaN,1.0,VC1,VC_1_2,VC,VC,5731,210.0,421.0,0.50,2.000000e-10
207,NaN,1.0,VC1,VC_1_3,VC,VC,9979,418.0,936.0,0.45,2.000000e-10
208,NaN,1.0,VC1,VC_1_4,VC,VC,13472,399.0,432.0,0.92,2.000000e-10
209,NaN,1.0,VC1,VC_1_5,VC,VC,6035,180.0,478.0,0.38,2.000000e-10
210,NaN,1.0,VC1,VC_1_6,VC,VC,7627,82.0,479.0,0.17,2.000000e-10
211,NaN,2.0,VC2,VC_2_1,VC,VC,9769,60.0,606.0,0.10,2.000000e-10
212,NaN,2.0,VC2,VC_2_2,VC,VC,19263,494.0,613.0,0.80,2.000000e-10
213,NaN,2.0,VC2,VC_2_3,VC,VC,12540,924.0,578.0,1.60,2.000000e-10
214,NaN,2.0,VC2,VC_2_4,VC,VC,15135,156.0,542.0,0.29,2.000000e-10


In [7]:
%connect_info

{
  "shell_port": 56155,
  "iopub_port": 56156,
  "stdin_port": 56157,
  "control_port": 56159,
  "hb_port": 56158,
  "ip": "127.0.0.1",
  "key": "393c016e-388664bc905ae05be699f3ed",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": "python3"
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-25da9361-7887-4782-ba80-3c1d89529e98.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
